Цель — обучить модель, которая принимает на вход строку без пробелов и восстанавливает корректное расположение пробелов. Для решения пользуемся ByT5-small, работающая на уровне байтов, гуд для обработки текстов без токенизации по словам. В обучение я взял данные Wikipedia RU (Hugging Face datasets) из исходных удалял пробелы отправлял на вход, оригинальный текст использовал как целевой выход, файнтюним ByT5-small на CPU 5к тренировочных пар одна эпоха. На инференсе генерируется текст с пробелами, а потом преобразуется в список индексов пробелов

In [ ]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import random
import torch
import regex as re
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)

In [ ]:
MODEL_NAME = "google/byt5-small"
OUTPUT_DIR = "./byt5-spaces"
MAX_LEN = 128
TRAIN_SAMPLES = 5000
NUM_EPOCHS = 1
BATCH_SIZE = 16       
LR = 5e-4
LOG_STEPS = 50

TEST_CSV = "dataset_1937770_3.txt"   #мой тестовый датасет 
SUBMISSION_CSV = "submission.csv"
SEED = 42

#несколько утилит 
def set_seed(seed=42): #воспроизводимость 
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
def normalize_whitespace(text: str) -> str: #удаление лишних пробелов
    return re.sub(r"\s+", " ", str(text)).strip()

In [ ]:
def build_pairs_from_texts(texts, max_samples=20000, max_len=128): #строим пары (вход без пробелов, цель с пробелами)
    pairs = []
    for t in texts:
        if not t or not isinstance(t, str):
            continue
        t = normalize_whitespace(t)
        if len(t) < 10 or " " not in t:
            continue
        step = max_len // 2 if max_len >= 8 else max_len
        for i in range(0, len(t), step):
            chunk = t[i:i + max_len]
            if len(chunk) < 5 or " " not in chunk:
                continue
            # Удаляем ВСЕ типы пробелов
            inp = re.sub(r"\s+", "", chunk)
            if len(inp) < 3:
                continue
            pairs.append({"input": inp, "target": chunk})
            if len(pairs) >= max_samples:
                return pairs
    return pairs

def preprocess_function(examples, tokenizer, max_len=128): #токенизация входа и таргета
    #токенизация входа
    model_inputs = tokenizer(
        examples["input"], 
        max_length=max_len, 
        truncation=True, 
        padding=False
    )
    #токенизация таргета
    labels = tokenizer(
        examples["target"], 
        max_length=max_len, 
        truncation=True, 
        padding=False
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def restore_spaces_batch(texts, model, tokenizer, max_len=128): #
    model.eval()
    device = next(model.parameters()).device
    inputs = tokenizer(
        texts, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=max_len
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_length=max_len, 
            num_beams=4   # улучшает результаты
        )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True) #возвращаем список текстов с пробелами

In [ ]:
#cид и принуждаем к CPU 
set_seed(SEED)
device = torch.device("cpu")
print("Device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
model.to(device)

Данные

In [ ]:
#загрузка википедии
wiki = load_dataset("wikimedia/wikipedia", "20231101.ru", split="train[:200000]") #загружаем википедию 

#формируем пары
pairs = build_pairs_from_texts( #формируем тренировочные пары 
    wiki["text"], 
    max_samples=TRAIN_SAMPLES, 
    max_len=MAX_LEN
)

df = pd.DataFrame(pairs)
train_ds = Dataset.from_pandas(df)

#токенизация
tokenized_train = train_ds.map(
    lambda ex: preprocess_function(ex, tokenizer, MAX_LEN),
    batched=True, 
    remove_columns=train_ds.column_names
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Трейним (около 12 минут)

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    save_strategy="no",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=LOG_STEPS,
    dataloader_num_workers=0, #иначе ложится 
    group_by_length=False,
    optim="adafactor",
    eval_strategy ="no",   
    fp16=False,                 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

#обучение
trainer.train()

Инферинсим (тоже около 10 минут)

In [ ]:
print("\nЗагрузка тестового файла:", TEST_CSV)
rows = []
with open(TEST_CSV, encoding="utf-8") as f: #там есть гадкие строки, где запятые внутри текста 
    header = f.readline().strip().split(",")  # ['id', 'text_no_spaces']
    for line in f:
        line = line.rstrip("\n")
        if not line:
            continue
        id_part, text_part = line.split(",", 1)  #делим только по первой запятой
        rows.append((id_part, text_part))

import pandas as pd
test_df = pd.DataFrame(rows, columns=header)
assert "text_no_spaces" in test_df.columns, "В тестовом CSV должен быть столбец 'text_no_spaces'."

texts = test_df["text_no_spaces"].astype(str).tolist()

print("Генерация результатов")
BATCH_GEN = 32
restored_all = []
for i in range(0, len(texts), BATCH_GEN):
    batch = texts[i:i+BATCH_GEN]
    restored_all.extend(restore_spaces_batch(batch, model, tokenizer, MAX_LEN))

test_df["restored_text"] = restored_all

NameError: name 'TEST_CSV' is not defined

In [ ]:
#получаем позиции пробелов
def get_space_positions(orig_text, restored_text):
    positions = []
    i, j = 0, 0
    while i < len(orig_text) and j < len(restored_text):
        if restored_text[j] == " ":
            # пробел в restored перед символом orig[i]
            positions.append(i)  
            j += 1
        else:
            i += 1
            j += 1
    return positions

test_df["predicted_positions"] = [
    str(get_space_positions(orig, restored))
    for orig, restored in zip(test_df["text_no_spaces"], test_df["restored_text"])
]

#сохраняем финальный submission
submission = test_df[["id", "predicted_positions"]]
submission.to_csv(SUBMISSION_CSV, index=False, encoding="utf-8-sig")

print("Сохранено:", SUBMISSION_CSV)
print(submission.head(10))

Сохранено: submission.csv
  id  predicted_positions
0  0          [5, 10, 12]
1  1            [3, 6, 7]
2  2  [4, 12, 13, 20, 21]
3  3          [5, 10, 18]
4  4           [2, 5, 10]
5  5           [6, 7, 13]
6  6              [5, 14]
7  7          [3, 12, 15]
8  8              [6, 13]
9  9                  [7]
